# Trabajo practico Nro 3, haciendo ciencia de datos

## Introduccion

En este informe intentaremos predecir la falta de pago de las tarjetas de credito de clientes de American Express. Para ello usaremos un dataset reducido del provisto por la empresa en [esta competencia](https://www.kaggle.com/competitions/amex-default-prediction/overview/description).

Para este trabajo primero vamos a reducir el [dataset original de entrenamiento](https://www.kaggle.com/competitions/amex-default-prediction/data?select=train_data.csv) en un 95% para poder analizarlo completamente desde Kaggle.

El dataset original cuenta con informacion del perfil de varios clientes dividido en 18 meses, cada registro del dataset es una abstraccion del resumen de la tarjeta de credito de un cliente para una fecha dada. y si el cliente no paga 120 dias despues de haber recibido su ultimo resumen se lo considera un cliente moroso.

Como nuestro dataset reducido solo considera un 5% de los registros del dataset original, no vamos a poder ver en detalle cada resumen de cada cliente, por lo que nuestro analisis se limitara a lo que podemos observar con el dataset reducido.

## Dependencias

Para el analisis de datos y prediccion de morosos, haremos uso de las siguientes bibliotecas

In [ ]:
!pip install visualkeras

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
import scipy.stats as st
import sklearn as sk
import visualkeras
import tensorflow as tf
import plotly.graph_objects as go
import xgboost as xgb
import math
import io
from math import pi
from tensorflow import keras
from sklearn import svm, preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, export_text
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, RandomizedSearchCV, GridSearchCV, cross_validate
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, f1_score, make_scorer,classification_report, precision_score
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import VotingClassifier
from plotly.subplots import make_subplots
from keras.models import Model
from keras.layers import Dense, Input, Dropout
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)


tf.random.set_seed(42)

## Anexo - Generacion del dataset reducido

Dejamos el algoritmo utilizado para la generacion del dataset reducido. Se hace uso de la funcion ***sample_without_replacement*** con un *random state* particular de nuestro grupo. Con ello mantenemos la proporcion original del dataset y nos permite realizar un analisis mas cercano al que se haria con el dataset original.

```py
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.utils.random import sample_without_replacement
from pathlib import Path

filename = '.../train_data.csv'

def row_count():
    with open(filename) as f:
        return sum(1 for line in f)

state = (31416 * 9) % 1000 #Grupo 9
count = row_count()
keep_rows = sample_without_replacement(n_population=count, n_samples=int(count * (0.05)), random_state=state)
keep_rows = np.insert(keep_rows, 0,0, axis=0)
data = pd.read_csv(filename, skiprows=lambda x: x not in keep_rows)
filepath = Path('.../reduced_data.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
data.to_csv(filepath)
```

Cargamos el dataset reducido y dropeamos la primer columna, que duplica la informacion del index.

In [ ]:
data = pd.read_csv('../input/amexreducido/reduced_data.csv')

data.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
data

Tambien cargamos el dataset de labels, con la variable objetivo **target**

| _Variable_              	| _Tipo_                	| _Descripción_                                                   	|
|-------------------------	|-----------------------	|-----------------------------------------------------------------	|
| Target             	| Cualitstiva binaria                      	| Indica si un cliente es moroso                                                 	|

In [ ]:
labels = pd.read_csv("../input/amex-default-prediction/train_labels.csv")
labels

## Ciencia de datos

### Introducción al dataset
A continuación exploraremos el dataset para tener una mejor comprensión de sus variables. Hacemos un merge entre los datos y la variable objetivo para analizarlas en conjunto.

In [ ]:
data_analysis = pd.merge(data, labels, on='customer_ID', how='inner')

**Variables del dataset segun su representacion**

| _Nomenclatura_	| _Representa_	| _Numeracion_                      	|
|------------------	|--------------	|-----------------------------------	|
|D_*            	| Delincuencia	| D_39 - D_145 (96 variables)       	|
|S_*            	| Gasto     	| S_2 - S_27 (22 variables)         	|
|P_*            	| Pago      	| P_2 - P_4 (3 variables)           	|
|B_*            	| Balance      	| B_1 - B_42 (40 variables)         	|
|R_*            	| Riesgo      	| R_1 - R_28 (28 variables)         	|
|               	|           	| **Total: 189 variables**           	|

In [ ]:
categories=['Delincuencia', 'Gasto','Pago','Balance','Riesgo']
values= [len(data.filter(regex='D_').columns), len(data.filter(regex='S_').columns),len(data.filter(regex='P_').columns), len(data.filter(regex='B_').columns),len(data.filter(regex='R_').columns)]


fig = go.Figure()
fig.add_trace(go.Pie(values = values,labels = categories,hole = 0.6, 
                     hoverinfo ='label+percent', textfont=dict(color="white")))
fig.update_layout(title='Distribución de las categorías de las variables', 
                  legend=dict(traceorder='reversed',y=1.05,x=0),
                  uniformtext_minsize=15, uniformtext_mode='hide',width=700)
fig.show()

**De las variables anteriormente descriptas, las siguientes son informadas como categoricas segun American Express**

| _Variable_    	| _Tipo de variable_	| _Valores_                |
|------------------	|-------------------	|--------------------------|
|B_30            	| Cualitativa ordinal	| {0, 1, 2}                |
|B_38            	| Cualitativa ordinal  	| {1, 2, 3, 4, 5, 6, 7}    |
|D_114            	| Cualitativa binaria  	| {0, 1}                   |
|D_116            	| Cualitativa binaria  	| {0, 1}                   |
|D_117            	| Cualitativa ordinal  	| {-1, 1, 2, 3, 4, 5, 6}   |
|D_120            	| Cualitativa binaria  	| {0, 1}                   |
|D_126             	| Cualitativa ordinal  	| {-1, 0, 1}               | 
|D_63             	| Cualitativa nominal  	| {CL, CO, CR, XL, XM, XZ} |
|D_64             	| Cualitativa nominal  	| {-1, O, R, U}            |
|D_66              	| Cualitativa binaria  	| {0, 1}                   |
|D_68             	| Cualitativa ordinal  	| {0, 1, 2, 3, 4, 5, 6}    |
|**Total: 10 variables**               	| | |

**Analisis de variables categoricas**

In [ ]:
categorical = data_analysis.nunique().sort_values(ascending=True).reset_index(name='count').head(15)

A continuacion vemos las 15 variables con menor cantidad de valores diferentes, notamos que hay algunas variables que tienen pocos valores pero que no estan informadas en el detalle del dataset como categoricas, de igual manera las analizaremos en conjunto.

In [ ]:
categorical

In [ ]:
categorical = categorical.head(14)
categorical_columns = categorical['index'].values
categorical

En los siguientes histogramas vemos la frecuencia de los valores de cada variable segun el target (en rojo estan los morosos y en azul los pagos). Observamos que las variables *D_87* y *B_31* tienen frecuencias particulares. Por un lado *D_87* tiene mayoria de valores nulos y *B_31* mayormente el valor 1

In [ ]:
categorical_ignore_target = [x for x in categorical_columns if x != "target"]
fig = make_subplots(rows=5, cols=3, 
                    subplot_titles=categorical_ignore_target, 
                    vertical_spacing=0.1)
pal, color=['#48D0FF','#A61B1B'], ['#6FD6FF','#C12323']
row=0
c=[1,2,3]*5
plot_df=data_analysis[categorical_columns]
for i,col in enumerate(categorical_ignore_target):
    if i%3==0:
        row+=1
    df=plot_df.replace(np.nan, 'NA').replace('', 'NA').groupby(col)['target'].value_counts().rename('count').reset_index()
    x_label = [str(int(item)) if isinstance(item, (int, float)) else str(item) for item in df[col].drop_duplicates().reset_index()[col].values]
    fig.add_trace(go.Bar(x=x_label, y=df[df.target==1]['count'],
                         marker_color=pal[1], marker_line=dict(color=pal[1],width=2), 
                         hovertemplate='Impagos. Frecuencia = %{y}',
                         name='Impago', showlegend=(True if i==0 else False)),
                  row=row, col=c[i])
    fig.add_trace(go.Bar(x=x_label, y=df[df.target==0]['count'],
                         marker_color=pal[0], marker_line=dict(color=pal[0],width=2),
                         hovertemplate='Pagos. Frecuencia = %{y}',
                         name='Pago', showlegend=(True if i==0 else False)),
                  row=row, col=c[i])
    if i%3==0:
        fig.update_yaxes(title='Frecuencia',row=row,col=c[i])
fig.update_layout(legend=dict(orientation="h",yanchor="bottom",y=1.03,xanchor="right",x=0.85),
                  barmode='group',height=1500,width=900)
fig.show()

**Analisis de la distribucion de resumenes por fecha**

En el siguiente grafico podemos observar como se distribuye el porcentaje de deudores y no deudores por fecha de resumen. Notamos que se trata de una distribucion casi uniforme, por lo que podemos considerar que la distribucion de resumenes de clientes deudores y no deudores es uniforme para todo el dataset, con valores cercanos al 80% de resumenes de clientes no deudores cada fecha

In [ ]:
target=pd.DataFrame(data={'Deudores':data_analysis.groupby('S_2')['target'].mean()*100})
target['Pagos']=np.abs(data_analysis.groupby('S_2')['target'].mean()-1)*100
fig=go.Figure()
fig.add_trace(go.Bar(x=target.index, y=target.Pagos, name='Pagos',
                     text=target.Pagos, marker=dict(color=color[0],line=dict(color=pal[0],width=1.5)),
                     hovertemplate = "<b>%{x}</b><br>Clientes pagos: %{y:.2f}%"))
fig.add_trace(go.Bar(x=target.index, y=target.Deudores, name='Deudores',
                     marker=dict(color=color[1],line=dict(color=pal[1],width=1.5)),
                     hovertemplate = "<b>%{x}</b><br>Default accounts: %{y:.2f}%"))
fig.update_layout(barmode='relative', yaxis_ticksuffix='%', width=1400,
                  legend=dict(orientation="h", traceorder="reversed", yanchor="bottom",y=1.1,xanchor="left", x=0))
fig.show()

**Analisis del balance de los datos**

Observamos que existe un desbalanceo de los datos con una tendencia a clientes que no tienen deudas. Esto ya se podia presuponer del grafico anterior, ya que cada dia se cuenta con mas resumenes de clientes no deudores que de clientes deudores.

In [ ]:
target=data_analysis.target.value_counts(normalize=True)
target.rename(index={1:'Deudores',0:'Pagos'},inplace=True)
pal, color=['#48D0FF','#A61B1B'], ['#6FD6FF','#C12323']
fig=go.Figure()
fig.add_trace(go.Pie(labels=target.index, values=target*100, hole=.45, 
                     showlegend=True,sort=False, 
                     marker=dict(colors=color,line=dict(color=pal,width=2.5)),
                     hovertemplate = "Clientes %{label}: %{value:.2f}%<extra></extra>"))
fig.update_layout(title='Balance de los datos', 
                  legend=dict(traceorder='reversed',y=1.05,x=0),
                  uniformtext_minsize=15, uniformtext_mode='hide',width=700)
fig.show()

**Análisis de las variables de pago**

Analizamos el dataset enfocandonos en las variables de pago.

In [ ]:
data_analysis_p = data_analysis.filter(regex='P_')
data_analysis_p.head()

In [ ]:
data_analysis_p.describe().applymap("{0:.2f}".format)

Distribucion de las variables de pago segun la situacion de la deuda impaga o paga de cada cliente. Notamos que la variable P_2 tiene distribuciones distintas cuando se trata de clientes deudores y pagos, mientras que P_3 y P_4 tienen distribuciones parecidas.

Esto puede indicarnos que P_2 puede llegar a ser un factor interesante para decidir la situación de la deuda.

In [ ]:
cols=[col for col in data_analysis.columns if (col.startswith(('P','t')))]
plot_df=data_analysis[cols]
fig, ax = plt.subplots(1,3, figsize=(16,5))
fig.suptitle('Distribucion de las variables de pago',fontsize=16)
for i, col in enumerate(plot_df.columns[:-1]):
    sns.kdeplot(x=col, hue='target', palette=pal[::-1], hue_order=[1,0], 
                label=['Impago','Pago'], data=plot_df, 
                fill=True, linewidth=2, legend=False, ax=ax[i])
    ax[i].tick_params(left=False,bottom=False)
    ax[i].set(title='{}'.format(col), xlabel='', ylabel=('Densidad' if i==0 else ''))
handles, _ = ax[0].get_legend_handles_labels() 
fig.legend(labels=['Impago','Pago'], handles=reversed(handles), ncol=2, bbox_to_anchor=(0.18, 1))
sns.despine(bottom=True, trim=True)
plt.tight_layout(rect=[0, 0.2, 1, 0.99])

Vemos tambien que P_4 se comporta casi como una variable discreta, tomando valores muy cercanos a 0 o muy cercanos a 1.

Ahora analizaremos la correlación entre las variables de pago y la variable objetivo

In [ ]:
corr=plot_df.corr()
mask=np.triu(np.ones_like(corr, dtype=bool))[1:,:-1]
corr=corr.iloc[1:,:-1].copy()
fig, ax = plt.subplots()   
sns.heatmap(corr, mask=mask, vmin=-1, vmax=1, center=0, annot=True, fmt='.2f', 
            cmap='coolwarm', annot_kws={'fontsize':12,'fontweight':'bold'}, cbar=False)
ax.tick_params(left=False,bottom=False)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right',fontsize=12)
ax.set_yticklabels(ax.get_yticklabels(), fontsize=12)
plt.title('Correlacion entre variables de pago\n', fontsize=16)
fig.show()

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(16,5))
fig.suptitle('Relaciones entre las variables de pago, transformacion logaritmica',fontsize=16)
ax[0].hexbin(x='P_2', y='P_3', data=plot_df, bins='log', gridsize=40, cmap='coolwarm')
ax[0].text(-.2,2.2, 'Correlation: {:.2f}'.format(plot_df[['P_2','P_3']].corr().iloc[1,0]), 
           ha="center", va="center",bbox=dict(boxstyle="round,pad=0.3",fc="white"))
ax[0].set(xlabel='P_2',ylabel='P_3')
ax[1].hexbin(x='P_3', y='P_4', data=plot_df, bins='log', gridsize=40, cmap='coolwarm')
ax[1].text(-.6,1.35, 'Correlation: {:.2f}'.format(plot_df[['P_3','P_4']].corr().iloc[1,0]), 
           ha="center", va="center",bbox=dict(boxstyle="round,pad=0.3",fc="white"))
ax[1].set(xlabel='P_3',ylabel='P_4')
ax[2].hexbin(x='P_4', y='P_2', data=plot_df, bins='log', gridsize=40, cmap='coolwarm')
ax[2].text(.25,1.1, 'Correlation: {:.2f}'.format(plot_df[['P_4','P_2']].corr().iloc[1,0]), 
           ha="center", va="center",bbox=dict(boxstyle="round,pad=0.3",fc="white"))
ax[2].set(xlabel='P_4',ylabel='P_2')
for i in range(3):
    ax[i].tick_params(left=False,bottom=False)
sns.despine()
plt.tight_layout(rect=[0, 0, 1, 0.99])
plt.show()

Analizamos los nulos y vemos que no representan una gran cantidad respecto a la totalidad del dataset

In [ ]:
null_p=round((data_analysis_p.isna().sum()/data_analysis_p.shape[0]*100),2).sort_values(ascending=False).astype(str)+('%')
null_p=null_p.to_frame().rename(columns={0:'Nulos'})
null_p

**Análisis de las variables de riesgo**

Analizamos el dataset enfocandonos en las variables de pago.

In [ ]:
data_analysis_r = data_analysis.filter(regex='R_')

In [ ]:
data_analysis_r.head()

Analizamos las distribuciones de variables para las de riesgo. De nuevo vemos distribuciones parecidas entre los clientes deudores y no deudores. Observamos tambien variables con valores atipicos muy marcados, con máximos muy alejados de su media y cuartiles.

In [ ]:
cols=[col for col in data_analysis.columns if (col.startswith(('R','t'))) & (col not in categorical_ignore_target)]
plot_df=data_analysis[cols]
fig, ax = plt.subplots(6,5, figsize=(16,24))
fig.suptitle('Distribucion de las variables de riesgo',fontsize=16)
row=0
col=[0,1,2,3,4]*6
for i, column in enumerate(plot_df.columns[:-1]):
    if (i!=0)&(i%5==0):
        row+=1
    sns.kdeplot(x=column, hue='target', palette=pal[::-1], hue_order=[1,0], 
                label=['Impago','Pago'], data=plot_df, 
                fill=True, linewidth=2, legend=False, ax=ax[row,col[i]])
    ax[row,col[i]].tick_params(left=False,bottom=False)
    ax[row,col[i]].set(title='\n\n{}'.format(column), xlabel='', ylabel=('Densidad' if i%5==0 else ''))
for i in range(3,5):
    ax[5,i].set_visible(False)
handles, _ = ax[0,0].get_legend_handles_labels() 
fig.legend(labels=['Impago','Pago'], handles=reversed(handles), ncol=2, bbox_to_anchor=(0.18, 0.984))
sns.despine(bottom=True, trim=True)
plt.tight_layout(rect=[0, 0.2, 1, 0.99])

In [ ]:
data_analysis_r[['R_5', 'R_7', 'R_8', 'R_14', 'R_26', 'R_20']].describe().applymap("{0:.2f}".format)

In [ ]:
corr=plot_df.corr()
mask=np.triu(np.ones_like(corr, dtype=bool))[1:,:-1]
corr=corr.iloc[1:,:-1].copy()
fig, ax = plt.subplots(figsize=(24,18))   
sns.heatmap(corr, mask=mask, vmin=-1, vmax=1, center=0, annot=True, fmt='.2f', 
            cmap='coolwarm', annot_kws={'fontsize':12,'fontweight':'bold'}, cbar=False)
ax.tick_params(left=False,bottom=False)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right',fontsize=12)
ax.set_yticklabels(ax.get_yticklabels(), fontsize=12)
plt.title('Correlaciones entre variables de riesgo\n', fontsize=16)
fig.show()

Encontramos correlaciones fuertes entre algunas variables (R_2/R_4, R_8/R_5), pero tambien vemos otras muy debiles (R_18, R_23, R_28) con el resto de variables.

Analizando los valores nulos para las variables de riesgo, encontramos R_9 y R_26 con porcentajes muy altos de registros sin informacion.

In [ ]:
null_r=round((data_analysis_r.isna().sum()/data_analysis_r.shape[0]*100),2).sort_values(ascending=False).astype(str)+('%')
null_r=null_r.to_frame().rename(columns={0:'Nulos'})
null_r.head(10)

**Análisis de las variables de balance**

Analizamos el dataset enfocandonos en las variables de balance.

In [ ]:
data_analysis_b = data_analysis.filter(regex='B_')

In [ ]:
data_analysis_b.head()

Analizamos las distribuciones de las variables de balance. De nuevo vemos distribuciones parecidas para deudores y no deudores, junto con variables con valores atipicos marcados.

In [ ]:
cols=[col for col in data_analysis.columns if (col.startswith(('B','t'))) & (col not in categorical_ignore_target)]
plot_df=data_analysis[cols]
fig, ax = plt.subplots(8,5, figsize=(16,32))
fig.suptitle('Distribucion de las variables de balance',fontsize=16)
row=0
col=[0,1,2,3,4]*8
for i, column in enumerate(plot_df.columns[:-1]):
    if (i!=0)&(i%5==0):
        row+=1
    sns.kdeplot(x=column, hue='target', palette=pal[::-1], hue_order=[1,0], 
                label=['Impago','Pago'], data=plot_df, 
                fill=True, linewidth=2, legend=False, ax=ax[row,col[i]])
    ax[row,col[i]].tick_params(left=False,bottom=False)
    ax[row,col[i]].set(title='\n\n{}'.format(column), xlabel='', ylabel=('Densidad' if i%5==0 else ''))
for i in range(3,5):
    ax[7,i].set_visible(False)
handles, _ = ax[0,0].get_legend_handles_labels() 
fig.legend(labels=['Impago','Paid'], handles=reversed(handles), ncol=2, bbox_to_anchor=(0.18, 0.984))
sns.despine(bottom=True, trim=True)
plt.tight_layout(rect=[0, 0.2, 1, 0.99])

In [ ]:
data_analysis_b[['B_6', 'B_10', 'B_12', 'B_13', 'B_14', 'B_15', 'B_26', 'B_40']].describe().applymap("{0:.2f}".format)

Analizamos la correlacion entre variables. En este caso encontramos correlaciones lineales mucho mas marcadas que con las variables anteriores. Graficamos algunas de ellas.

In [ ]:
corr=plot_df.corr()
mask=np.triu(np.ones_like(corr, dtype=bool))[1:,:-1]
corr=corr.iloc[1:,:-1].copy()
fig, ax = plt.subplots(figsize=(24,22))   
sns.heatmap(corr, mask=mask, vmin=-1, vmax=1, center=0, annot=True, fmt='.2f', 
            cmap='coolwarm', annot_kws={'fontsize':12,'fontweight':'bold'}, cbar=False)
ax.tick_params(left=False,bottom=False)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right',fontsize=12)
ax.set_yticklabels(ax.get_yticklabels(), fontsize=12)
plt.title('Correlaciones entre variables de balance\n', fontsize=16)
fig.show()

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(16,5))
fig.suptitle('Relaciones entre las variables de balance, transformacion logaritmica',fontsize=16)
ax[0].hexbin(x='B_37', y='B_1', data=plot_df, bins='log', gridsize=40, cmap='coolwarm')
ax[0].text(-2.2,1.1, 'Correlacion: {:.2f}'.format(plot_df[['B_37','B_1']].corr().iloc[1,0]), 
           ha="center", va="center",bbox=dict(boxstyle="round,pad=0.3",fc="white"))
ax[0].set(xlabel='B_37',ylabel='B_1')
ax[1].hexbin(x='B_37', y='B_11', data=plot_df, bins='log', gridsize=40, cmap='coolwarm')
ax[1].text(-2.2,1.6, 'Correlacion: {:.2f}'.format(plot_df[['B_37','B_11']].corr().iloc[1,0]), 
           ha="center", va="center",bbox=dict(boxstyle="round,pad=0.3",fc="white"))
ax[1].set(xlabel='B_37',ylabel='B_11')
ax[2].hexbin(x='B_23', y='B_7', data=plot_df, bins='log', gridsize=40, cmap='coolwarm')
ax[2].text(.25,1.1, 'Correlacion: {:.2f}'.format(plot_df[['B_23','B_7']].corr().iloc[1,0]), 
           ha="center", va="center",bbox=dict(boxstyle="round,pad=0.3",fc="white"))
ax[2].set(xlabel='B_23',ylabel='B_7')
for i in range(3):
    ax[i].tick_params(left=False,bottom=False)
sns.despine()
plt.tight_layout(rect=[0, 0, 1, 0.99])
plt.show()

Encontramos a las variables B_39, B_42, B_29 y B_17 con porcentajes elevados de registros sin informacion.

In [ ]:
null_b=round((data_analysis_b.isna().sum()/data_analysis_b.shape[0]*100),2).sort_values(ascending=False).astype(str)+('%')
null_b=null_b.to_frame().rename(columns={0:'Nulos'})
null_b.head(10)

**Análisis de las variables de gasto**

Analizamos el dataset enfocandonos en las variables de gasto.

In [ ]:
data_analysis_s = data_analysis.filter(regex='S_')

In [ ]:
data_analysis_s.head()

Viendo la distribución de las variables respecto a la variable objetivo, observamos el mismo comportamiento que con las variables ya analizadas.

In [ ]:
cols=[col for col in data_analysis.columns if (col.startswith(('S','t'))) & (col != 'S_2')]
plot_df=data_analysis[cols]
fig, ax = plt.subplots(5,5, figsize=(16,20))
fig.suptitle('Distribucion de las variables de gasto',fontsize=16)
row=0
col=[0,1,2,3,4]*5
for i, column in enumerate(plot_df.columns[:-1]):
    if (i!=0)&(i%5==0):
        row+=1
    sns.kdeplot(x=column, hue='target', palette=pal[::-1], hue_order=[1,0], 
                label=['Impago','Pago'], data=plot_df, 
                fill=True, linewidth=2, legend=False, ax=ax[row,col[i]])
    ax[row,col[i]].tick_params(left=False,bottom=False)
    ax[row,col[i]].set(title='\n\n{}'.format(column), xlabel='', ylabel=('Densidad' if i%5==0 else ''))
for i in range(1,5):
    ax[4,i].set_visible(False)
handles, _ = ax[0,0].get_legend_handles_labels() 
fig.legend(labels=['Impago','Pago'], handles=reversed(handles), ncol=2, bbox_to_anchor=(0.18, 0.985))
sns.despine(bottom=True, trim=True)
plt.tight_layout(rect=[0, 0.2, 1, 0.99])

In [ ]:
data_analysis_s[['S_5', 'S_12', 'S_16', 'S_22', 'S_23', 'S_24', 'S_26']].describe().applymap("{0:.2f}".format)

Observando la correlación entre las distintas variables de gasto, sobresalen 2 relaciones fuertes, que separaremos para visualizar más de cerca.

In [ ]:
corr=plot_df.corr()
mask=np.triu(np.ones_like(corr, dtype=bool))[1:,:-1]
corr=corr.iloc[1:,:-1].copy()
fig, ax = plt.subplots(figsize=(24,22))   
sns.heatmap(corr, mask=mask, vmin=-1, vmax=1, center=0, annot=True, fmt='.2f', 
            cmap='coolwarm', annot_kws={'fontsize':12,'fontweight':'bold'}, cbar=False)
ax.tick_params(left=False,bottom=False)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right',fontsize=12)
ax.set_yticklabels(ax.get_yticklabels(), fontsize=12)
plt.title('Correlaciones entre variables de gasto\n', fontsize=16)
fig.show()

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(12,5))
fig.suptitle('Relaciones entre las variables de balance, transformacion logaritmica',fontsize=16)
ax[0].hexbin(x='S_7', y='S_3', data=plot_df, bins='log', gridsize=40, cmap='coolwarm')
ax[0].text(0,5, 'Correlacion: {:.2f}'.format(plot_df[['S_7','S_3']].corr().iloc[1,0]), 
           ha="center", va="center",bbox=dict(boxstyle="round,pad=0.3",fc="white"))
ax[0].set(xlabel='S_7',ylabel='S_3')
ax[1].hexbin(x='S_24', y='S_22', data=plot_df, bins='log', gridsize=40, cmap='coolwarm')
ax[1].text(-80,0, 'Correlacion: {:.2f}'.format(plot_df[['S_24','S_22']].corr().iloc[1,0]), 
           ha="center", va="center",bbox=dict(boxstyle="round,pad=0.3",fc="white"))
ax[1].set(xlabel='S_24',ylabel='S_22')
for i in range(2):
    ax[i].tick_params(left=False,bottom=False)
sns.despine()
plt.tight_layout(rect=[0, 0, 1, 0.99])
plt.show()

Analizando los registros nulos, si bien no son tan elevados como en la variable anterior, S_9 supera el 50% del dataset.

Otro dato interesante es que tanto S_7 como S_3 presentan el mismo porcentaje de nulos, y de los gráficos de correlación habíamos observado que se encontraban fuertemente relacionados, por lo tanto tiene sentido que los nulos provengan de la misma fuente de datos.

In [ ]:
null_s=round((data_analysis_s.isna().sum()/data_analysis_s.shape[0]*100),2).sort_values(ascending=False).astype(str)+('%')
null_s=null_s.to_frame().rename(columns={0:'Nulos'})
null_s.head(10)

**Análisis de las variables de delincuencia**

Analizamos el dataset enfocandonos en las variables de delincuencia.

In [ ]:
data_analysis_d = data_analysis.filter(regex='D_')

In [ ]:
data_analysis_d.head()

Volvemos a observar distribuciones parecidas respecto al resto de las variables, donde no se ve de forma marcada que una variable de delincuencia contribuya a la detección de una deuda impaga. Para D_88 por ejemplo si bien se observa mayor cantidad de deudas impagas, no nos alcanza para asumir que en la totalidad de los casos donde esto ocurra, la deuda será impaga.

In [ ]:
cols=[col for col in data_analysis.columns if (col.startswith(('D','t'))) & (col not in categorical_ignore_target)]
plot_df=data_analysis[cols]
fig, ax = plt.subplots(18,5, figsize=(16,54))
fig.suptitle('Distribucion de las variables de delincuencia',fontsize=16)
row=0
col=[0,1,2,3,4]*18
for i, column in enumerate(plot_df.columns[:-1]):
    if (i!=0)&(i%5==0):
        row+=1
    sns.kdeplot(x=column, hue='target', palette=pal[::-1], hue_order=[1,0], 
                label=['Impago','Pago'], data=plot_df, 
                fill=True, linewidth=2, legend=False, ax=ax[row,col[i]])
    ax[row,col[i]].tick_params(left=False,bottom=False)
    ax[row,col[i]].set(title='\n\n{}'.format(column), xlabel='', ylabel=('Densidad' if i%5==0 else ''))
for i in range(2,5):
    ax[17,i].set_visible(False)
handles, _ = ax[0,0].get_legend_handles_labels() 
fig.legend(labels=['Impago','Pago'], handles=reversed(handles), ncol=2, bbox_to_anchor=(0.18, 0.984))
sns.despine(bottom=True, trim=True)
plt.tight_layout(rect=[0, 0.2, 1, 0.99])

Analizando la correlación, vemos correlaciones marcadas entre diferentes variables. Hacemos zoom en 3 correlaciones particulares:

In [ ]:
corr=plot_df.corr()
mask=np.triu(np.ones_like(corr, dtype=bool))[1:,:-1]
corr=corr.iloc[1:,:-1].copy()
fig, ax = plt.subplots(figsize=(60,60))   
sns.heatmap(corr, mask=mask, vmin=-1, vmax=1, center=0, annot=True, fmt='.2f', 
            cmap='coolwarm', annot_kws={'fontsize':12,'fontweight':'bold'}, cbar=False)
ax.tick_params(left=False,bottom=False)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right',fontsize=12)
ax.set_yticklabels(ax.get_yticklabels(), fontsize=12)
plt.title('Correlaciones entre variables de delincuencia\n', fontsize=16)
fig.show()

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(12,5))
fig.suptitle('Relaciones entre las variables de delincuencia, transformacion logaritmica',fontsize=16)
ax[0].hexbin(x='D_75', y='D_74', data=plot_df, bins='log', gridsize=40, cmap='coolwarm')
ax[0].text(1,4, 'Correlacion: {:.2f}'.format(plot_df[['D_75','D_74']].corr().iloc[1,0]), 
           ha="center", va="center",bbox=dict(boxstyle="round,pad=0.3",fc="white"))
ax[0].set(xlabel='D_75',ylabel='D_74')
ax[1].hexbin(x='D_77', y='D_62', data=plot_df, bins='log', gridsize=40, cmap='coolwarm')
ax[1].text(2,11, 'Correlacion: {:.2f}'.format(plot_df[['D_77','D_62']].corr().iloc[1,0]), 
           ha="center", va="center",bbox=dict(boxstyle="round,pad=0.3",fc="white"))
ax[1].set(xlabel='D_77',ylabel='D_62')
ax[2].hexbin(x='D_119', y='D_118', data=plot_df, bins='log', gridsize=40, cmap='coolwarm')
ax[2].text(0.5,1.8, 'Correlacion: {:.2f}'.format(plot_df[['D_119','D_118']].corr().iloc[1,0]), 
           ha="center", va="center",bbox=dict(boxstyle="round,pad=0.3",fc="white"))
ax[2].set(xlabel='D_119',ylabel='D_118')
for i in range(3):
    ax[i].tick_params(left=False,bottom=False)
sns.despine()
plt.tight_layout(rect=[0, 0, 1, 0.99])
plt.show()

A diferencia del resto de las variables, para las de delincuencia se observa una cantidad bastante más marcada de nulos, donde hay 14 columnas con más del 90% de nulos.

In [ ]:
null_d=round((data_analysis_d.isna().sum()/data_analysis_d.shape[0]*100),2).sort_values(ascending=False).astype(str)+('%')
null_d=null_d.to_frame().rename(columns={0:'Nulos'})
null_d.head(30)

### Análisis y limpieza de valores nulos

Si bien ya vimos los nulos separados por cada tipo de variable, vamos a realizar un análisis más general de los nulos del dataset para decidir que hacer con ellos.

In [ ]:
nulls=round((data_analysis.isna().sum()/data_analysis.shape[0]*100),2).sort_values(ascending=False)
nulls=nulls.to_frame().rename(columns={0:'Nulos (%)'})
nulls.head(30)

Dada la cantidad de columnas con valores nulos, eliminaremos aquellas con un porcentaje mayor al 80% porque de los valores restantes, si los completamos con un valor (la media o la mediana), no nos daría mucha información de la columna. Y si eliminamos los registros nulos, perderíamos casi la totalidad del dataset.

In [ ]:
nulls_cols = nulls[nulls['Nulos (%)'] >= 80]
nulls_cols

In [ ]:
data.drop(columns = nulls_cols.index.values, inplace = True)

In [ ]:
data

Guardamos todas las variables con nulos en una lista para analizarlos

In [ ]:
analisis_nulos = nulls_cols.index.values
analisis_nulos

Vemos si para las variables que quedaron alguna esta muy correlacionada con otra para poder borrarla

In [ ]:
analisis_cols = [x for x in analisis_nulos if (x not in categorical_ignore_target)]
data_columns = [x for x in data_analysis.columns.values if (x != 'customer_ID') & (x != 'S_2') & (x not in categorical_ignore_target) & (x not in analisis_cols)]
#Create a new dictionary
plotDict = {}
# Loop across each of the two lists that contain the items you want to compare
for gene1 in list(analisis_cols):
    for gene2 in list(data_columns):
        df_nonulls = data_analysis[[gene1, gene2]].dropna(axis=0)
        if(len(df_nonulls[gene1]) < 2):
            continue
        # Do a pearsonR comparison between the two items you want to compare
        tempDict = {(gene1, gene2): 
                    scipy.stats.pearsonr(df_nonulls[gene1],df_nonulls[gene2])
                    }
        # Update the dictionary each time you do a comparison
        plotDict.update(tempDict)
# Unstack the dictionary into a DataFrame
dfOutput = pd.Series(plotDict, dtype='float64').unstack()
# Optional: Take just the pearsonR value out of the output tuple
corr = dfOutput.apply(lambda x: x.apply(lambda x:x[0]))

Debido a que es una gran cantidad de variables las que queremos comparar, vamos a separar el heatmap en dos partes

In [ ]:
mid = int(len(corr.columns) / 2)
corr_a = corr[corr.columns[0:mid]]
corr_b = corr[corr.columns[mid:]]
fig, ax = plt.subplots(2, figsize=(32,12))
fig.suptitle('Correlaciones entre variables eliminadas y el resto del dataset', fontsize=16)
sns.heatmap(corr_a, vmin=-1, vmax=1, center=0, square=True, linewidth=0.01, 
            cmap='coolwarm', cbar=False, ax=ax[0])
ax[0].tick_params(left=False,bottom=False)
ax[0].set_xticklabels(ax[0].get_xticklabels(), horizontalalignment='right',fontsize=8)
ax[0].set_yticklabels(ax[0].get_yticklabels(), fontsize=8)
sns.heatmap(corr_b, vmin=-1, vmax=1, center=0, square=True, linewidth=0.01, 
            cmap='coolwarm', cbar=False, ax=ax[1])
ax[1].tick_params(left=False,bottom=False)
ax[1].set_xticklabels(ax[1].get_xticklabels(), horizontalalignment='right',fontsize=8)
ax[1].set_yticklabels(ax[1].get_yticklabels(), fontsize=8)
fig.show()

Definimos un valor de correlación como threshold de 0.7 para aquellas que podemos borrar.

In [ ]:
threshold = .5
delete_spectre = .7
corr_columns = [corr.columns[j] for i, j in zip(*np.where(np.abs(corr.values) >= delete_spectre))]
corr_elevadas = [f"{corr.index[i]} y {corr.columns[j]} = {corr.iloc[i][j]:.2f}" for i, j in zip(*np.where(np.abs(corr.values) >= threshold))]
print("\n" + f"Columnas a eliminar (corr > {delete_spectre}): " + ", ".join(corr_columns) + "\n")
print("Columnas con correlaciones elevadas: " + ", ".join(corr_elevadas))

In [ ]:
fig, ax = plt.subplots(2,2, figsize=(8,8))
cols=[col for col in data_analysis.columns if (col.startswith(('B','D'))) & (col not in categorical_ignore_target)]
plot_df=data_analysis[cols]
fig.suptitle('Relaciones entre las variables de delincuencia, transformacion logaritmica',fontsize=16)
ax[0][0].hexbin(x='D_131', y='D_132', data=plot_df, bins='log', gridsize=40, cmap='coolwarm')
ax[0][0].text(1.1,6, 'Correlacion: {:.2f}'.format(plot_df[['D_131','D_132']].corr().iloc[1,0]), 
           ha="center", va="center",bbox=dict(boxstyle="round,pad=0.3",fc="white"))
ax[0][0].set(xlabel='D_131',ylabel='D_132')
ax[0][1].hexbin(x='D_142', y='D_141', data=plot_df, bins='log', gridsize=40, cmap='coolwarm')
ax[0][1].text(0.5, 1.3, 'Correlacion: {:.2f}'.format(plot_df[['D_142','D_141']].corr().iloc[1,0]), 
           ha="center", va="center",bbox=dict(boxstyle="round,pad=0.3",fc="white"))
ax[0][1].set(xlabel='D_142',ylabel='D_141')
ax[1][0].hexbin(x='B_17', y='B_39', data=plot_df, bins='log', gridsize=40, cmap='coolwarm')
ax[1][0].text(0.2, 1.15, 'Correlacion: {:.2f}'.format(plot_df[['B_17','B_39']].corr().iloc[1,0]), 
           ha="center", va="center",bbox=dict(boxstyle="round,pad=0.3",fc="white"))
ax[1][0].set(xlabel='B_17',ylabel='B_39')
ax[1][1].hexbin(x='D_59', y='D_111', data=plot_df, bins='log', gridsize=40, cmap='coolwarm')
ax[1][1].text(0.4, 0.8, 'Correlacion: {:.2f}'.format(plot_df[['D_59','D_111']].corr().iloc[1,0]), 
           ha="center", va="center",bbox=dict(boxstyle="round,pad=0.3",fc="white"))
ax[1][1].set(xlabel='D_59',ylabel='D_111')

for i in range(2):
    for j in range(2):
        ax[i][j].tick_params(left=False,bottom=False)
sns.despine()
plt.tight_layout(rect=[0, 0, 1, 0.99])
plt.show()

In [ ]:
data.drop(columns = corr_columns, inplace = True)

In [ ]:
data

Luedo de haber tratado las de > 80% y filtrado las que están muy correlacionadas, vemos qué variables quedaron con nulos:

In [ ]:
fill_nulls=round((data.isna().sum()/data.shape[0]*100),2).sort_values(ascending=False)
fill_nulls=fill_nulls.to_frame().rename(columns={0:'Nulos (%)'})
fill_nulls.head(30)

De todas las variables la mayoría presenta una baja cantidad de nulos; para las que presentan un porcentaje importante ser verá de llenar esos nulos.

In [ ]:
fill_nulls=fill_nulls[fill_nulls['Nulos (%)'] > 10]
fill_nulls

Vemos las medidas de resumen para esas columnas:

In [ ]:
data[fill_nulls.index.values].describe().applymap(lambda x: f"{x:0.2f}")

Vemos que la desviación estandar es baja para todas las variables (< 0.7) por lo que decidimos llenar los nulos de cada una con sus medianas.

In [ ]:
data[fill_nulls.index.values] = data[fill_nulls.index.values].fillna(data[fill_nulls.index.values].median())

In [ ]:
remain_nulls = round((data.isna().sum()/data.shape[0]*100),2).sort_values(ascending=False)
remain_nulls=remain_nulls.to_frame().rename(columns={0:'Nulos (%)'})
remain_nulls.head(30)

Vemos que como resultado de todo el análisis de las columans con nulos, las columnas que nos quedaron con nulos sin tratar representan un porcentaje muy bajo. Para cada modelo luego vamos a decidir qué hacer con estos nulos.

### Análisis de outliers

Comenzaremos el análisis graficando un boxplot para cada variable del dataset. Utilizaremos un boxplot ya que permite ver para todas las variables al mismo tiempo de forma práctica si hay valores atípicos.  
El boxplot permite realizar un análisis univariado de los datos para cada columna, resultando en posibles outliers moderados aquellos que se encuentran entre la caja (espacio entre el primer y tercer cuartil) y los bigotes (mínimo/máximo), y outliers severos aquellos que se encuentran más allá de los bigotes.

In [ ]:
# Comentado con el fin de incrementar la velocidad con la que ejecuta todo
data.boxplot(figsize=(150,15))

Dado el problema planteado, en donde hay que predecir si un cliente pagará o no, creemos que no tiene sentido eliminar todos los outliers ya que son los que pueden aportar la información más importante. Por lo tanto analizaremos las variables que tengan mayor presencia de outliers. Haciendo zoom en este boxplot se puede ver que son: B_6, B_10, D_69, R_7, B_26, S_16, R_14, S_26 y B_40.


In [ ]:
analisis_outliers = ['B_6', 'B_10', 'D_69', 'R_7', 'B_26', 'S_16', 'R_14', 'S_26', 'B_40']

Para seguir con el análisis univariado sobre cada una de las variables seleccionadas utilizaremos el método de la métrica Z-Score. Esta métrica se basa en cuan desviado está un dato de la media, asumiendo una distribución gaussiana, resultando los posibles outliers aquellos que den con módulo mayor a 3.

In [ ]:
z_cols = []
for col in analisis_outliers:
    z_col = 'z_' + col
    media = np.mean(data[col])
    std = np.std(data[col])
    data[z_col] = (data[col] - media) / std
    z_cols.append(z_col)

Así quedan los resultados para cada variable de la métrica de Z-Score

In [ ]:
data[z_cols]

A continuación verificamos cuántos registros tienen valores mayores a 3 (en módulo)

In [ ]:
sns.boxplot(y=data['B_6'])

In [ ]:
data[(abs(data['z_B_6']) >= 3)][['B_6', 'z_B_6']].sort_values(by='z_B_6', ascending=False)

Dado que los 580 registros representan sólo el 0.21% del total decidimos eliminarlos.

In [ ]:
data.drop(data[(abs(data['z_B_6']) >= 3)].index, inplace=True)

In [ ]:
sns.boxplot(y=data['B_10'])

In [ ]:
data[(abs(data['z_B_10']) >= 3)][['B_10', 'z_B_10']].sort_values(by='z_B_10', ascending=False)

Dado que los 172 registros representan sólo el 0.062% del total decidimos eliminarlos.

In [ ]:
data.drop(data[(abs(data['z_B_10']) >= 3)].index, inplace=True)

In [ ]:
sns.boxplot(y=data['D_69'])

In [ ]:
data[(abs(data['z_D_69']) >= 3)][['D_69', 'z_D_69']].sort_values(by='z_D_69', ascending=False)

Dado que los 73 registros representan sólo el 0.026% del total decidimos eliminarlos.

In [ ]:
data.drop(data[(abs(data['z_D_69']) >= 3)].index, inplace=True)

In [ ]:
sns.boxplot(y=data['R_7'])

In [ ]:
data[(abs(data['z_R_7']) >= 3)][['R_7', 'z_R_7']].sort_values(by='z_R_7', ascending=False)

Dado que los 918 registros representan sólo el 0.332% del total decidimos eliminarlos.

In [ ]:
data.drop(data[(abs(data['z_R_7']) >= 3)].index, inplace=True)

In [ ]:
sns.boxplot(y=data['B_26'])

In [ ]:
data[(abs(data['z_B_26']) >= 3)][['B_26', 'z_B_26']].sort_values(by='z_B_26', ascending=False)

Dado que los 370 registros representan sólo el 0.134% del total decidimos eliminarlos.

In [ ]:
data.drop(data[(abs(data['z_B_26']) >= 3)].index, inplace=True)

In [ ]:
sns.boxplot(y=data['S_16'])

In [ ]:
data[(abs(data['z_S_16']) >= 3)][['S_16', 'z_S_16']].sort_values(by='z_S_16', ascending=False)

Dado que los 1475 registros representan sólo el 0.533% del total decidimos eliminarlos.

In [ ]:
data.drop(data[(abs(data['z_S_16']) >= 3)].index, inplace=True)

In [ ]:
sns.boxplot(y=data['R_14'])

In [ ]:
data[abs(data['z_R_14']) >= 3][['R_14', 'z_R_14']].sort_values(by='z_R_14', ascending=False)

Dado que los 432 registros representan sólo el 0.156% del total decidimos eliminarlos.

In [ ]:
sns.boxplot(y=data['S_26'])

In [ ]:
data[abs(data['z_S_26']) >= 3][['S_26', 'z_S_26']].sort_values(by='z_S_26', ascending=False)

Dado que los 1235 registros representan sólo el 0.447% del total decidimos eliminarlos.

In [ ]:
data.drop(data[abs(data['z_S_26']) >= 3].index, inplace=True)

In [ ]:
sns.boxplot(y=data['B_40'])

In [ ]:
data[abs(data['z_B_40']) >= 3][['B_40', 'z_B_40']].sort_values(by='z_B_40', ascending=False)

Dado que los 259 registros representan sólo el 0.094% del total decidimos eliminarlos.

In [ ]:
data.drop(data[abs(data['z_B_40']) >= 3].index, inplace=True)

Vemos cómo quedó el dataset finalmente

In [ ]:
data

De los 276572 registros originales quedaron 271490, lo que equivale al 98.16% del dataset. 

Eliminamos las columnas que agregamos para evaluar los outliers.

In [ ]:
data.drop(columns=z_cols, inplace=True)

In [ ]:
data

### Columnas a agregar

Revisamos si hay datos que aporten información como para crear columnas nuevas.

De las columnas que estan representadas en forma de texto, nos fijamos qué información presentan:

In [ ]:
cols_txt = data.select_dtypes(include=['object']).columns
cols_txt

In [ ]:
data[cols_txt]

Customer ID es el identificador del deudor al cual queremos clasificar. S_2 representa una fecha la cual podemos dividir en 3 variables numéricas (año, mes y día), mientras que D_63 y D_64 son strings que representan diferentes categorías.

Crearemos una columna para el año, el mes y el día extraidos de la columna S_2

In [ ]:
data['S_2'] = pd.to_datetime(data['S_2']) 
data['S_2_Year'] = data['S_2'].dt.year
data['S_2_Month'] = data['S_2'].dt.month
data['S_2_Day'] = data['S_2'].dt.day

In [ ]:
data[['S_2', 'S_2_Year', 'S_2_Month', 'S_2_Day']]

Borramos S_2 asi no duplicamos informacion

In [ ]:
data.drop(columns=['S_2'], inplace=True)

### Reducción de dimensionalidad

In [ ]:
data.shape

Dado que hay muchas columnas vemos si es posible reducir la cantidad de columnas para mejorar los tiempos de ejecución. Para ello utilizaremos PCA (Análisis de Componentes Principales).

In [ ]:
from sklearn.decomposition import PCA

Dado que sólo funciona con variables numéricas filtraremos las columnas que cumplan con este requisito

In [ ]:
cols = data.select_dtypes(include=['number']).columns
cols = [x for x in cols if (x not in ['S_2_Year', 'S_2_Month', 'S_2_Day'])]
not_cols = [x for x in data.columns if (x not in cols)]
not_cols

In [ ]:
data_parte_a = data[not_cols].copy()
data_parte_b = data[cols].copy()

In [ ]:
nulos=round((data_parte_b.isna().sum()/data_parte_b.shape[0]*100),5).sort_values(ascending=False)
nulos=nulos.to_frame().rename(columns={0:'Nulos (%)'})
nulos = nulos[nulos['Nulos (%)'] > 0]
nulos

Dado que PCA no permite las variables con nulos, vamos a llenar los nulos con la mediana de cada variable dado que los porcentajes son chicos y no se modificará de una forma importante su distribución.

In [ ]:
for col in nulos.index.values:
    data_parte_b[col] = data_parte_b[col].fillna(data_parte_b[col].median())

In [ ]:
tipos_de_variables = "DSPBR"
for tipo in tipos_de_variables:
    var = tipo + '_'
    print(f"{var}: {len(data_parte_b.filter(regex=var).columns)}")

In [ ]:
data_d = data_parte_b.filter(regex="D_").copy()
data_s = data_parte_b.filter(regex="S_").copy()
data_b = data_parte_b.filter(regex="B_").copy()
data_r = data_parte_b.filter(regex="R_").copy()
data_p = data_parte_b.filter(regex="P_").copy()

Al momento de reducir la dimensionalidad, buscaremos quedarnos con las variables que cubran el 97.5% de la varianza ya que de quedarnos con el 95% podríamos perder valores _"borde"_ que aporten buena información para este problema puntual.

Como las variables respectivas a la delincuencia son las más abundantes, empezaremos por reducir esas variables.

In [ ]:
pca_data = PCA()
pca_data.fit(data_d)

Calculamos la cantidad de componentes principales a utilizar

In [ ]:
var_cumu = np.cumsum(pca_data.explained_variance_ratio_) * 100

In [ ]:
k = np.argmax(var_cumu > 97.5)
print("El numero minimo de componentes para explicar el 97.5% de la varianza es: " + str(k))

In [ ]:
plt.figure(figsize=[10, 5])
plt.title('Varianza acumulada explicada por componente')
plt.ylabel('Varianza acumulada explicada')
plt.xlabel('Componentes principales')
plt.axvline(x=k, color="k", linestyle="--")
plt.axhline(y=97.5, color="r", linestyle="--")
ax = plt.plot(var_cumu)
plt.show()

Continuamos con la cantidad de componentes principales que nos dan el 97.5% de variabilidad explicada

In [ ]:
pca = PCA(n_components=k)
pca_transform=pca.fit_transform(data_d)

In [ ]:
columnas = ["D_PCA_" + f"{i}" for i in range(41)]
pca_d = pd.DataFrame(data = pca_transform, columns=columnas)

In [ ]:
pca_d

Repetimos el proceso ahora para las variables respectivas al gasto

In [ ]:
pca_data = PCA()
pca_data.fit(data_s)

Calculamos la cantidad de componentes principales a utilizar

In [ ]:
var_cumu = np.cumsum(pca_data.explained_variance_ratio_) * 100

k = np.argmax(var_cumu > 97.5)
print("El numero minimo de componentes para explicar el 97.5% de la varianza es: " + str(k))

In [ ]:
plt.figure(figsize=[10, 5])
plt.title('Varianza acumulada explicada por componente')
plt.ylabel('Varianza acumulada explicada')
plt.xlabel('Componentes principales')
plt.axvline(x=k, color="k", linestyle="--")
plt.axhline(y=97.5, color="r", linestyle="--")
ax = plt.plot(var_cumu)
plt.show()

Continuamos con la cantidad de componentes principales que nos dan el 97.5% de variabilidad explicada

In [ ]:
pca = PCA(n_components=k)
pca_transform=pca.fit_transform(data_s)

columnas = ["S_PCA_" + f"{i}" for i in range(14)]
pca_s = pd.DataFrame(data = pca_transform, columns=columnas)

In [ ]:
pca_s

Repetimos el proceso ahora para las variables respectivas al balance

In [ ]:
pca_data = PCA()
pca_data.fit(data_b)

Calculamos la cantidad de componentes principales a utilizar

In [ ]:
var_cumu = np.cumsum(pca_data.explained_variance_ratio_) * 100

k = np.argmax(var_cumu > 97.5)
print("El numero minimo de componentes para explicar el 97.5% de la varianza es: " + str(k))

In [ ]:
plt.figure(figsize=[10, 5])
plt.title('Varianza acumulada explicada por componente')
plt.ylabel('Varianza acumulada explicada')
plt.xlabel('Componentes principales')
plt.axvline(x=k, color="k", linestyle="--")
plt.axhline(y=97.5, color="r", linestyle="--")
ax = plt.plot(var_cumu)
plt.show()

Continuamos con la cantidad de componentes principales que nos dan el 95% de variabilidad explicada

In [ ]:
pca = PCA(n_components=k)
pca_transform=pca.fit_transform(data_b)

columnas = ["B_PCA_" + f"{i}" for i in range(19)]
pca_b = pd.DataFrame(data = pca_transform, columns=columnas)

In [ ]:
pca_b

Repetimos el proceso ahora para las variables respectivas al riesgo

In [ ]:
pca_data = PCA()
pca_data.fit(data_r)

Calculamos la cantidad de componentes principales a utilizar

In [ ]:
var_cumu = np.cumsum(pca_data.explained_variance_ratio_) * 100

k = np.argmax(var_cumu > 97.5)
print("El numero minimo de componentes para explicar el 97.5% de la varianza es: " + str(k))

Como no tiene sentido eliminar variables, nos vamos a quedar con 5.

In [ ]:
k = 5

In [ ]:
plt.figure(figsize=[10, 5])
plt.title('Varianza acumulada explicada por componente')
plt.ylabel('Varianza acumulada explicada')
plt.xlabel('Componentes principales')
plt.axvline(x=k, color="k", linestyle="--")
plt.axhline(y=97.5, color="r", linestyle="--")
ax = plt.plot(var_cumu)
plt.show()

In [ ]:
pca = PCA(n_components=k)
pca_transform=pca.fit_transform(data_r)

columnas = ["R_PCA_" + f"{i}" for i in range(5)]
pca_r = pd.DataFrame(data = pca_transform, columns=columnas)

In [ ]:
pca_r

Ahora unimos el dataset

In [ ]:
datasets = [data_parte_a, pca_d, pca_s, pca_b, pca_r, data_p]
for d in datasets:
    d.reset_index(inplace=True)
nuevo = pd.concat(datasets, axis=1)
nuevo.drop(columns=['index'], inplace=True)

In [ ]:
nuevo

In [ ]:
data

Nos quedamos con el nuevo dataset

In [ ]:
data = nuevo

**Normalización de los datos**  
Evaluamos la posible normalizacion de los datos para cada variable, para eso veremos la antidad de valores que son mayores a 1.

In [ ]:
contador_total = 0
contador_variables = 0
for col in data.columns:
    contador = 0
    for elemento in data[col]:
        contador_total += 1
        if(isinstance(elemento, float) and elemento > 1):
            contador += 1
            contador_variables += 1
    print(f"{col}: {contador}")
print(f"Porcentaje total: {round(contador_variables/contador_total, 2)}" + "%")

In [ ]:
data

In [ ]:
data = data.reindex(sorted(data.columns), axis=1)

In [ ]:
data

In [ ]:
for col in data.columns:
    if(col not in ["customer_ID", "S_2_Year", "S_2_Month", "S_2_Day"]):
        for elemento in data[col]:
            if(isinstance(elemento, float) and elemento > 1):
                data[col]=(data[col]-data[col].min())/(data[col].max()-data[col].min())
    
data

In [ ]:
data

### Tareas de limpieza y transformación extras

Vamos a pasar a numeros todas las variables del dataset para no tener problemas con los distintos modelos. Aquellas que no puedan pasarse no serán tenidas en cuenta al momento de dividir el set en train y test.

In [ ]:
cols = data.select_dtypes(include=['number']).columns
not_cols = [x for x in data.columns if (x not in cols)]
not_cols

Como habíamos visto en la sección Columnas a agregar, D_63 y D_64 eran columnas con información de tipo Object (string). Veamos cuántos valores pueden tomar

In [ ]:
print(f'D_63 toma {data["D_63"].nunique()} valores posibles')
print(f'D_64 toma {data["D_64"].nunique()} valores posibles')

Son pocos los valores categorícos que toma, por lo tanto decidimos aplicar One Hot Encoding por medio de get_dummies, y modelamos los nulos como una categoría más:

In [ ]:
data = pd.get_dummies(data, columns=['D_64'], dummy_na=True)

In [ ]:
data = pd.get_dummies(data, columns=['D_63'], dummy_na=True)

In [ ]:
data

### Balanceo del dataset

Hasta el momento se trabajó con la parte de datos del dataset, ahora haremos un merge con los labels para ver qué tan balanceado se encuentra

In [ ]:
completo = pd.merge(data, labels, on='customer_ID', how='inner')
completo

In [ ]:
colores=["#75201a","#194373"]
completo['target'].value_counts().plot.bar(alpha=0.9, color=colores)
plt.show()

Vemos que el dataset está desbalanceado, hay mucha cantidad de registros para los cuales no se hizo el pago (target == 0).

Es por ello que vamos a armar un segundo dataset con un balanceo por cliente manteniendo una proporcion 70 / 30 entre clientes no deudores y clientes deudores

In [ ]:
cant_total = completo.shape[0]
cant_clientes = len(completo['customer_ID'].unique())
cant_clientes_morosos = len(completo[completo['target'] == 1]['customer_ID'].unique())
cant_clientes_pagos = len(completo[completo['target'] == 0]['customer_ID'].unique())
cant_target1 = completo[completo['target'] == 1].shape[0]
cant_target0 = completo[completo['target'] == 0].shape[0]
cant_elim = cant_target0 - cant_target1
ratio = 70/30

clientes_unicos = completo.drop_duplicates(subset=['customer_ID'])[['customer_ID', 'target']]
sample_clientes_unicos_pagos = clientes_unicos[clientes_unicos['target']==0].sample(n=math.ceil(cant_clientes_morosos * ratio), random_state=3)

sample_clientes_pagos = completo[completo['customer_ID'].isin(sample_clientes_unicos_pagos['customer_ID'].values)]
clientes_morosos = completo[completo['target']==1]

completo_balanceado = pd.concat([sample_clientes_pagos, clientes_morosos])

cant_quedan_clientes = len(completo_balanceado['customer_ID'].unique())
cant_quedan = completo_balanceado.shape[0]

print(f"\nEl dataset tiene {cant_total} registros y {cant_clientes} clientes, de los cuales " + \
      f"{cant_target1} registros corresponden a {cant_clientes_morosos} clientes target 1 y " + \
      f"{cant_target0} registros corresponden a {cant_clientes_pagos} clientes target 0")
print(f"\nEl dataset balanceado quedaría con {math.ceil(cant_clientes_morosos * ratio)} (cantidad de clientes target 0) + {cant_clientes_morosos} " + \
      f"(cantidad de clientes target 1) = {cant_quedan_clientes} clientes " + \
      f"y {cant_quedan} registros")

In [ ]:
completo_balanceado

In [ ]:
colores=["#75201a","#194373"]
completo_balanceado['target'].value_counts().plot.bar(alpha=0.9, color=colores)

### Definicion de metricas y prediccion por cliente

Como podemos observar existe mas de un resumen de tarjeta de credito por cliente en nuestro dataset. Ademas, como solo consideramos un 5% del mismo, no tenemos los 18 resumenes de cada cliente, por lo que algunos van a tener mas resumenes que otros, pero con un mismo target por cliente.

Para nuestro analisis vamos a definir metricas de precision, recall y F1 agrupadas por cliente, en donde cada resumen se predice de forma individual, para luego quedarnos con la media de cada cliente.

Tambien debemos agregar un split del dataset por cliente para mantener una proporcion de clentes conocida y tener todos los resumenes de un cliente en el mismo split (ya sea para entrenar el modelo o para predecir)

Por ultimo vamos a agregar variables que puedan relacionar resumenes de un mismo cliente. Para ello vamos a entrenar un clasificador que nos pueda indicar cuales fueron las variables mas influyentes en su prediccion y luego nos quedamos con una media por cliente de esas variables y las usamos para predecir con el resto de modelos.

In [ ]:
from functools import reduce

def custom_predict(serie):
    return 0 if ((reduce(lambda x, y: x + y, serie)) / len(serie)) < 0.5 else 1

def split_por_cliente(balanceado, desbalanceado, porcentaje):
    balanceado_customer_target = balanceado[['customer_ID', 'target']]
    balanceado_clientes_unicos = balanceado.drop_duplicates(subset=['customer_ID'])[['customer_ID', 'target']]
    morosos_unicos = balanceado_customer_target[balanceado_customer_target["target"]==1]["customer_ID"].unique()
    pagos_unicos = balanceado_customer_target[balanceado_customer_target["target"]==0]["customer_ID"].unique()
    cantidad_morosos = math.ceil(len(morosos_unicos) * porcentaje)
    cantidad_pagos = math.ceil(len(pagos_unicos) * porcentaje)
    morosos_unicos_porcentaje = balanceado_clientes_unicos[balanceado_clientes_unicos['target']==1].sample(n=cantidad_morosos, random_state=3)
    pagos_unicos_porcentaje = balanceado_clientes_unicos[balanceado_clientes_unicos['target']==0].sample(n=cantidad_pagos, random_state=3)
    train_morosos = balanceado[balanceado['customer_ID'].isin(morosos_unicos_porcentaje['customer_ID'].values)]
    train_pagos = balanceado[balanceado['customer_ID'].isin(pagos_unicos_porcentaje['customer_ID'].values)]
    train_final = pd.concat([train_morosos, train_pagos])
    test_final = desbalanceado[~desbalanceado['customer_ID'].isin(train_final['customer_ID'].values)]
    y_train_final = train_final['target']
    y_test_final = test_final['target']
    return train_final.drop(['target'], axis='columns', inplace=False), test_final.drop(['target'], axis='columns', inplace=False), y_train_final, y_test_final
    
def metrics_by_client(real, predict):
    test_by_customer_predict = x_test_copy.copy()
    test_by_customer_predict['target'] = predict
    test_by_customer_predict = test_by_customer_predict[['customer_ID', 'target']].groupby('customer_ID').agg({'target':custom_predict})
    
    test_by_customer_real = x_test_copy.copy()
    test_by_customer_real['target'] = real
    test_by_customer_real = test_by_customer_real[['customer_ID', 'target']].groupby('customer_ID').agg({'target':custom_predict})

    acc = accuracy_score(test_by_customer_real['target'].values, test_by_customer_predict['target'].values)
    recall = recall_score(test_by_customer_real['target'].values, test_by_customer_predict['target'].values, average="weighted")
    f1 = f1_score(test_by_customer_real['target'].values, test_by_customer_predict['target'].values, average="weighted")
    
    return acc, recall, f1

def confusion_matrix_by_client(real, predict):
    test_by_customer_predict = x_test_copy.copy()
    test_by_customer_predict['target'] = predict
    test_by_customer_predict = test_by_customer_predict[['customer_ID', 'target']].groupby('customer_ID').agg({'target':custom_predict})
    
    test_by_customer_real = x_test_copy.copy()
    test_by_customer_real['target'] = real
    test_by_customer_real = test_by_customer_real[['customer_ID', 'target']].groupby('customer_ID').agg({'target':custom_predict})
    
    return confusion_matrix(test_by_customer_real['target'].values, test_by_customer_predict['target'].values)

**Split de conjunto de pruebas y entrenamiento**

A continuacion creamos los sets de entrenamiento y prueba. En este caso usaremos un 40% del dataset balanceado, por lo que se obtiene un dataset con el 40% de clientes target 1 del dataset original reducido, y el resto son clientes target 0 que completan una la misma proporcion que el dataset _completo_balanceado_.

El set de pruebas contiene el resto de clientes que no fueron utilizados para entrenar, por lo que vamos a poder analizar cada modelo con la totalidad del dataset original.

In [ ]:
x_train, x_test, y_train, y_test = split_por_cliente(completo_balanceado, completo, 0.4)

x_train_copy = x_train.copy()
x_test_copy = x_test.copy()

In [ ]:
unicos_train = len(x_train['customer_ID'].unique())
x_train_target = x_train
x_train_target['target'] = y_train
unicos_morosos_train = len(x_train_target[x_train_target['target']==1]['customer_ID'].unique())
unicos_pagos_train = len(x_train_target[x_train_target['target']==0]['customer_ID'].unique())
registros_train = x_train.shape[0]
registros_test = x_test.shape[0]
registros_balanceado = completo_balanceado.shape[0]
registros_completo = completo.shape[0]

print(f"\nLa cantidad de clientes unicos en train es: {unicos_train}")
print(f"La cantidad de clientes morosos unicos en train es: {unicos_morosos_train}, (Porcentaje del total: {round((unicos_morosos_train / unicos_train) * 100, 2)}%)")
print(f"La cantidad de clientes pagos unicos en train es: {unicos_pagos_train}, (Porcentaje del total: {round((unicos_pagos_train / unicos_train) * 100, 2)}%)")
print(f"La cantidad de registros en train es: {registros_train}, (Porcentaje respecto de balanceado: {round((registros_train / registros_balanceado) * 100, 2)}%, porcentaje respecto del total: {round((registros_train / registros_completo) * 100, 2)}%")
print(f"La cantidad de registros en test es: {registros_test}, (Porcentaje del total: {round((registros_test / registros_completo) * 100, 2)}%)")

**Analisis de variables mas influyentes**

Entrenamos el clasificador que nos va a indicar que variables fueron las que tuvieron mas influencia durante la prediccion, en este caso usaremos Random Forest

In [ ]:
model = RandomForestClassifier()
x_train_sin_customer = x_train.drop(['customer_ID', 'target'], axis='columns', inplace=False)
model.fit(x_train_sin_customer,y_train)
feat_importances = pd.Series(model.feature_importances_, index=x_train_sin_customer.columns)
feat_importances.nlargest(10).plot(kind='barh')
plt.show()

Prediccion con el conjunto de prueba balanceado

In [ ]:
x_test_sin_customer = x_test.drop(['customer_ID'], axis='columns', inplace=False)

predicted_categories = model.predict(x_test_sin_customer)

bf_acc, bf_recall, bf_f1 = metrics_by_client(y_test, predicted_categories)

print("La precision es {}".format(bf_acc))

print("El recall es {}".format(bf_recall))

print("F1 es {}".format(bf_f1))

Observamos la matriz de confusion para tener mas detalle sobre la prediccion de clientes deudores y no deudores.

In [ ]:
balanceado_important_columns_cm = confusion_matrix_by_client(y_test, predicted_categories)
sns.heatmap(balanceado_important_columns_cm, cmap='Blues', annot=True, fmt='g')
plt.xlabel('Predicted')
plt.ylabel('True')

Nos quedamos con las siguientes variables, que fueron las que mas influyeron con diferencia respecto del resto.

In [ ]:
columnas_importantes = ["B_PCA_0","P_2"]

Para las columnas con mayor importancia, agregamos una nueva feature por columna con la media agrupada por cliente

In [ ]:
completo_medias = completo.copy()
completo_balanceado_medias = completo_balanceado.copy()

for column in columnas_importantes:
    completo_medias[column + "_mean"] = (completo_medias.groupby("customer_ID")[column].transform('mean'))
    completo_balanceado_medias[column + "_mean"] = (completo_balanceado_medias.groupby("customer_ID")[column].transform('mean'))

In [ ]:
completo_medias

In [ ]:
completo_balanceado_medias

Dividimos al dataset con las columnas nuevas y creamos dos dataset para testear cada modelo entrenado. En uno guardamos el conjunto de test agrupado por cliente y el target real de cada uno, en el otro vamos a guardar una copia del conjunto de pruebas sin la columna de target, para completarla luego con las predicciones y tomar la media del target predicho por cliente (si la media es menor a 0.5 se considera target = 0, si no se considera target = 1)

In [ ]:
x_train, x_test, y_train, y_test = split_por_cliente(completo_balanceado_medias, completo_medias, 0.4)

x_train_copy = x_train.copy()
x_test_copy = x_test.copy()

Para hacer las pruebas de cada modelo, vamos a considerar en nuestro conjunto de test a todos los registros que no consideramos en el set de entrenamiento, y vamos a entrenar cada modelo con el set balanceado a un ratio de 60-40 (target 0 (%) - target 1 (%))

In [ ]:
#test_by_customer_real['target'].value_counts().plot.bar(alpha=0.9, color=colores)

Vamos a realizar una transformación en el dataset de forma que los datos tengan la forma de una desviación estandar.

In [ ]:
#Armo una version estandarizada
stand_scaler = preprocessing.StandardScaler()
x_train = stand_scaler.fit_transform(x_train.drop(['customer_ID'], axis='columns', inplace=False))
x_test = stand_scaler.fit_transform(x_test.drop(['customer_ID'], axis='columns', inplace=False))

## Generación y evaluación de modelos

Vamos a buscar los mejores hiperparámetros con KFOLD CV Random Search, usaremos sólo 2 folds ya que el dataset es muy grande y puede demorar demasiado tiempo.

#### Random Forest

Buscamos los mejores hiperparámetros.

In [ ]:
%%time
##KFOLD CV Random Search para buscar el mejor arbol (los mejores atributos, hiperparametros,etc)
from sklearn.model_selection import StratifiedKFold, KFold,RandomizedSearchCV
from sklearn.metrics import make_scorer

n=10

#Conjunto de parámetros que quiero usar
params_grid = {'criterion':['gini','entropy'],
               'min_samples_leaf':list(range(1,10)),
               'max_depth':list(range(3,6))}
                
#Cantidad de splits para el Cross Validation
folds=2

#Kfold estratificado
kfoldcv = StratifiedKFold(n_splits=folds)

#Clasificador
base_tree = RandomForestClassifier() 

#Metrica que quiero optimizar F1 Score
scorer_fn = make_scorer(sk.metrics.f1_score)

#Random Search Cross Validation
randomcv = RandomizedSearchCV(estimator=base_tree,
                              param_distributions=params_grid,
                              scoring=scorer_fn,
                              cv=kfoldcv,
                              n_iter=n,
                              random_state=420) 

#Busco los hiperparamtros que optimizan F1 Score
randomcv.fit(x_train,y_train);

#Mejores hiperparametros del arbol
print(randomcv.best_params_)
#Mejor métrica
print(randomcv.best_score_)

In [ ]:
# para no tener que correr random search siempre
params = {'min_samples_leaf': 3, 'max_depth': 5, 'criterion': 'entropy'}

Entrenamos el clasificador con pesos que permitan ajustar mejor el target 0 y los mejores hiperparametros.

In [ ]:
#Creo el árbol
#arbol = RandomForestClassifier(random_state=420).set_params(**randomcv.best_params_)
arbol = RandomForestClassifier(random_state=420).set_params(**params)

n=10

#Entreno el arbol en todo el set
arbol.fit(x_train,y_train)

predicted_categories1 = arbol.predict(x_test)

rf_acc1, rf_recall1, rf_f11 = metrics_by_client(y_test, predicted_categories)

print("La precision es {}".format(rf_acc1))

print("El recall es {}".format(rf_recall1))

print("F1 es {}".format(rf_f11))

Observamos que las columnas que agregamos se estan teniendo en cuenta por los arboles de clasificacion.

In [ ]:
feat_importances_rfc = pd.Series(arbol.feature_importances_, index=x_train_copy.drop(['customer_ID'], axis='columns', inplace=False).columns)
feat_importances_rfc.nlargest(10).plot(kind='barh')
plt.show()

Luego la prediccion por customer 

In [ ]:
rfc_cm = confusion_matrix_by_client(y_test, predicted_categories1)
sns.heatmap(rfc_cm, cmap='Blues', annot=True, fmt='g')
plt.xlabel('Predicted')
plt.ylabel('True')

#### XGBoost

Buscamos los mejores hiperparámetros

In [ ]:
%%time
##KFOLD CV Random Search para buscar el mejor clasificador (los mejores atributos, hiperparametros,etc)

#Conjunto de parámetros que quiero usar
params_grid = {'n_estimators':list(range(15,25)),
               'max_depth':list(range(6,10)),
               'eval_metric':list([["auc", "error"], ["auc", "error", "error@0.6"]])}
                
#Cantidad de splits para el Cross Validation
folds=2

#Kfold estratificado
kfoldcv = StratifiedKFold(n_splits=folds)

#Clasificador
base_model = xgb.XGBClassifier() 

#Metrica que quiero optimizar F1 Score
scorer_fn = make_scorer(sk.metrics.f1_score)

#Random Search Cross Validation
randomcv = RandomizedSearchCV(estimator=base_model,
                              param_distributions=params_grid,
                              scoring=scorer_fn,
                              cv=kfoldcv,
                              n_iter=n,
                              random_state=420) 

#Busco los hiperparamtros que optimizan F1 Score
randomcv.fit(x_train,y_train);

#Mejores hiperparametros del arbol
print(randomcv.best_params_)
#Mejor métrica
print(randomcv.best_score_)

In [ ]:
# para no tener que correr random search siempre
params = {'n_estimators': 22, 'max_depth': 6, 'eval_metric': ['auc', 'error']}

Creamos el modelo.

In [ ]:
%%time
#xgb_model = xgb.XGBClassifier(random_state=420).set_params(**randomcv.best_params_)
xgb_model = xgb.XGBClassifier(random_state=420).set_params(**params)

xgb_model.fit(x_train, y_train)

predicted_categories2 = xgb_model.predict(x_test)

rf_acc2, rf_recall2, rf_f12 = metrics_by_client(y_test, predicted_categories2)

print("La precision es {}".format(rf_acc2))

print("El recall es {}".format(rf_recall2))

print("F1 es {}".format(rf_f12))

In [ ]:
train_features = x_train_copy.drop(['customer_ID'], axis='columns', inplace=False).columns
feature_names = [train_features[int(x[1:])] for x in xgb_model.get_booster().get_score(importance_type='gain').keys()]
feat_importances_xgb = pd.Series(list(xgb_model.get_booster().get_score(importance_type='gain').values()), index=feature_names)
feat_importances_xgb.nlargest(10).plot(kind='barh')
plt.show()

In [ ]:
xgb_cm = confusion_matrix_by_client(y_test, predicted_categories2)
sns.heatmap(xgb_cm, cmap='Blues', annot=True, fmt='g')
plt.xlabel('Predicted')
plt.ylabel('True')

#### SVM

Buscamos los mejores hiperparámetros.

In [ ]:
%%time
##KFOLD CV Random Search para buscar el mejor clasificador (los mejores atributos, hiperparametros,etc)
n=4
#Conjunto de parámetros que quiero usar
params_grid = {'fit_intercept': list([True, False]),
               'dual': list([True, False])}
                
#Cantidad de splits para el Cross Validation
folds=2

#Kfold estratificado
kfoldcv = StratifiedKFold(n_splits=folds)

#Clasificador
base_model = svm.LinearSVC() 

#Metrica que quiero optimizar F1 Score
scorer_fn = make_scorer(sk.metrics.f1_score)

#Random Search Cross Validation
randomcv = RandomizedSearchCV(estimator=base_model,
                              param_distributions=params_grid,
                              scoring=scorer_fn,
                              cv=kfoldcv,
                              n_iter=n,
                              random_state=420) 

#Busco los hiperparamtros que optimizan F1 Score
randomcv.fit(x_train,y_train);

#Mejores hiperparametros del arbol
print(randomcv.best_params_)
#Mejor métrica
print(randomcv.best_score_)

In [ ]:
# para no tener que correr random search siempre
params = {'fit_intercept': False, 'dual': False, 'class_weight': {0: 1.5, 1: 0.6}}

Creamos el modelo.

In [ ]:
%%time
# Usamos svm.LinearSVC ya que está hecho para datasets grandes
#svm_model = svm.LinearSVC(random_state=420).set_params(**randomcv.best_params_)
svm_model = svm.LinearSVC(random_state=420).set_params(**params)

svm_model.fit(x_train, y_train)

predicted_categories3 = svm_model.predict(x_test)

rf_acc3, rf_recall3, rf_f13 = metrics_by_client(y_test, predicted_categories3)

print("La precision es {}".format(rf_acc3))

print("El recall es {}".format(rf_recall3))

print("F1 es {}".format(rf_f13))

In [ ]:
from sklearn.inspection import permutation_importance

perm_importance = permutation_importance(svm_model, x_test, y_test)

feature_names = x_train_copy.drop(['customer_ID'], axis='columns', inplace=False).columns
features = np.array(feature_names)

sorted_idx = perm_importance.importances_mean.argsort()[::-1][:10]
plt.barh(features[sorted_idx], perm_importance.importances_mean[sorted_idx])

In [ ]:
svm_cm = confusion_matrix_by_client(y_test, predicted_categories3)
sns.heatmap(svm_cm, cmap='Blues', annot=True, fmt='g')
plt.xlabel('Predicted')
plt.ylabel('True')

### Ensamble

Realizamos un ensamble de tipo VotingClassifier.

In [ ]:
%%time
#Creo ensemble de Votación
vot_clf = VotingClassifier(estimators = [('RF', arbol), ('xGBoost', xgb_model), ('SVM', svm_model)], voting = 'hard')

#Entreno el ensamble
vot_clf.fit(x_train, y_train)

predicted_categoriesE = vot_clf.predict(x_test)

rf_accE, rf_recallE, rf_f1E = metrics_by_client(y_test, predicted_categoriesE)

print("La precision es {}".format(rf_accE))

print("El recall es {}".format(rf_recallE))

print("F1 es {}".format(rf_f1E))

In [ ]:
cm = confusion_matrix_by_client(y_test, predicted_categoriesE)
sns.heatmap(cm, cmap='Blues', annot=True, fmt='g')
plt.xlabel('Predicted')
plt.ylabel('True')

### Red neuronal

Vamos a empezar por un modelo de red neuronal básico, y luego le vamos a ir agregando complejidad hasta llegar a un modelo predictor que supere a los anteriores

In [ ]:
columnas_predictoras=completo.drop(columns=['target', 'customer_ID']).columns.to_list()
d_in=len(columnas_predictoras)

Definimos funciones que nos van a ayudar a comparar a los diferentes modelos

In [ ]:
def plot_hist(history):
    acc = history.history['accuracy']
    loss = history.history['loss']
    
    epochs = range(1, len(acc) + 1)

    plt.plot(epochs, acc, 'b', label='Training acc')
    plt.title('Training accuracy')
    plt.legend()

    plt.figure()

    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.title('Training loss')
    plt.legend()

    plt.show()
    
def print_score(y_test, pred_red):
    
    red_acc, red_recall, red_f1 = metrics_by_client(y_test, pred_red)

    print("La precision es {}".format(red_acc))

    print("El recall es {}".format(red_recall))

    print("F1 es {}".format(red_f1))

El primer modelo que vamos a probar es uno básico, con 1 hidden layer de 16 nodos cuya función de regularización L2 es de tipo RELU. Como capa de output, elegimos una función de activación sigmoidea porque queremos clasificar 2 clases excluyentes. Como optimizador elegimos inicialmente SGD

In [ ]:
def baseline_red_neuronal():
    features = completo_medias.drop(['customer_ID', 'target'], axis='columns', inplace=False).columns.to_list()
    regularization = 4e-4
    activation_func = 'relu'
    inputs = Input(shape = (len(features)))

    x = Dense(16,
           kernel_regularizer = tf.keras.regularizers.l2(regularization),
           activation = activation_func)(inputs)

    x = Dense(1,
              activation='sigmoid')(x)

    model = Model(inputs, x)
    model.compile(
        optimizer=tf.keras.optimizers.SGD(),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=['accuracy'],
    )

    
    return model

In [ ]:
baseline_red = baseline_red_neuronal()
baseline_red.summary()

In [ ]:
%%time
# Entrenamiento del modelo
history_base = baseline_red.fit(x_train,y_train,epochs=10,batch_size=8,verbose=True)

Como la función de activación que estamos utilizando en la capa de salida es una función sigmoidea, el output de la red va a ser un número entre 0 y 1. Es por ello que vamos a aproximar según esté más cercano a 0 o a 1 para definir la clase a la que pertenece (pago o impago)

In [ ]:
pred_base_red = baseline_red.predict(x_test)

print_score(y_test, pred_base_red)

In [ ]:
rn1_cm = confusion_matrix_by_client(y_test, pred_base_red)
sns.heatmap(rn1_cm, cmap='Blues', annot=True, fmt='g')
plt.xlabel('Predicted')
plt.ylabel('True')

In [ ]:
plot_hist(history_base)

Vamos a partir de la misma base del modelo anterior pero ahora modificando el optimizador por Adam, con los valores default (LR 0,001, Beta_1 0,9 y Beta_2 0,999)

In [ ]:
def baseline_red_neuronal_v2():
    features = completo_medias.drop(['customer_ID', 'target'], axis='columns', inplace=False).columns.to_list()
    regularization = 4e-4
    activation_func = 'relu'
    inputs = Input(shape = (len(features)))

    x = Dense(16,
           kernel_regularizer = tf.keras.regularizers.l2(regularization),
           activation = activation_func)(inputs)

    x = Dense(1,
              activation='sigmoid')(x)

    model = Model(inputs, x)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=['accuracy'],
    )

    
    return model

In [ ]:
baseline_red_v2 = baseline_red_neuronal_v2()
baseline_red_v2.summary()

In [ ]:
%%time
# Entrenamiento del modelo
history_base_v2 = baseline_red_v2.fit(x_train,y_train,epochs=10,batch_size=8,verbose=True)

In [ ]:
pred_base_red_v2 = baseline_red_v2.predict(x_test)

print_score(y_test, pred_base_red)

In [ ]:
rn2_cm = confusion_matrix_by_client(y_test, pred_base_red_v2)
sns.heatmap(rn2_cm, cmap='Blues', annot=True, fmt='g')
plt.xlabel('Predicted')
plt.ylabel('True')

In [ ]:
plot_hist(history_base_v2)

Probamos otro modelo agregando una hidden layer después de la input layer, y sumando una capa de dropout entre ambas hidden layers para evitar el overfitting

In [ ]:
def red_neuronal_2_hidden_1_drop():
    features = completo_medias.drop(['customer_ID', 'target'], axis='columns', inplace=False).columns.to_list()
    regularization = 4e-4
    activation_func = 'relu'
    inputs = Input(shape = (len(features)))
    
    x = Dense(32,
           kernel_regularizer = tf.keras.regularizers.l2(regularization),
           activation = activation_func)(inputs)
    
    x = Dropout(0.1)(x)


    x = Dense(16,
           kernel_regularizer = tf.keras.regularizers.l2(regularization),
           activation = activation_func)(x)

    x = Dense(1,
              activation='sigmoid')(x)

    model = Model(inputs, x)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=['accuracy'],
    )

    
    return model

In [ ]:
red_2h1d = red_neuronal_2_hidden_1_drop()
red_2h1d.summary()

In [ ]:
%%time
# Entrenamiento del modelo
history_2h1d = red_2h1d.fit(x_train,y_train,epochs=10,batch_size=8,verbose=True)

In [ ]:
pred_red_2h1d = red_2h1d.predict(x_test)
print_score(y_test, pred_red_2h1d)

In [ ]:
rn3_cm = confusion_matrix_by_client(y_test, pred_base_red_v2)
sns.heatmap(rn3_cm, cmap='Blues', annot=True, fmt='g')
plt.xlabel('Predicted')
plt.ylabel('True')

In [ ]:
plot_hist(history_2h1d)

Probamos con un modelo similar, pero con menos nodos en las hidden layers

In [ ]:
def red_neuronal_2_hidden_1_drop_v2():
    features = completo_medias.drop(['customer_ID', 'target'], axis='columns', inplace=False).columns.to_list()
    regularization = 4e-4
    activation_func = 'relu'
    inputs = Input(shape = (len(features)))
    
    x = Dense(16,
           kernel_regularizer = tf.keras.regularizers.l2(regularization),
           activation = activation_func)(inputs)
    
    x = Dropout(0.1)(x)


    x = Dense(8,
           kernel_regularizer = tf.keras.regularizers.l2(regularization),
           activation = activation_func)(x)

    x = Dense(1,
              activation='sigmoid')(x)

    model = Model(inputs, x)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=['accuracy'],
    )

    
    return model

In [ ]:
red_2h1d_v2= red_neuronal_2_hidden_1_drop_v2()
red_2h1d_v2.summary()

También, a diferencia del resto de los modelos, esta vez entrenamos con más iteraciones (epochs 20 en lugar de 10) y un batch_size mayor (32 en lugar de 8). La idea es entrenar durante más iteraciones y aprovechar la memoria de la máquina para agarrar una muestra más grande para recorrer toda la red.

In [ ]:
%%time
# Entrenamiento del modelo
history_2h1d_v2 = red_2h1d_v2.fit(x_train,y_train,epochs=20,batch_size=32,verbose=True)

In [ ]:
pred_red_2h1d_v2 = red_2h1d_v2.predict(x_test)

print_score(y_test, pred_red_2h1d_v2)

In [ ]:
rn4_cm = confusion_matrix_by_client(y_test,pred_red_2h1d_v2)
sns.heatmap(rn4_cm, cmap='Blues', annot=True, fmt='g')
plt.xlabel('Predicted')
plt.ylabel('True')

In [ ]:
plot_hist(history_2h1d_v2)

Nuevamente volvemos a generar un modelo con una configuración similar, pero luego vamos a cambiar parámetros al entrenar el modelo.

In [ ]:
def red_neuronal(x_train, y_train):
    features = completo_medias.drop(['customer_ID', 'target'], axis='columns', inplace=False).columns.to_list()
    regularization = 4e-4
    activation_func = 'relu'
    inputs = Input(shape = (len(features)))
    
    x = Dense(32,
           kernel_regularizer = tf.keras.regularizers.l2(regularization),
           activation = activation_func)(inputs)
    
    x = Dropout(0.1)(x)

    x = Dense(16,
              kernel_regularizer=tf.keras.regularizers.l2(regularization),
              activation=activation_func)(x)

    x = Dense(1,
              activation='sigmoid')(x)

    model = Model(inputs, x)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=['accuracy'],
    )

    
    return model

In [ ]:
model_nr = red_neuronal(x_train, y_train)
model_nr.summary()

In [ ]:
visualkeras.layered_view(model_nr,legend=True) 

Vamos a agregar una función de callback de Early Stopping: si después de 5 iteraciones, la métrica de pérdida se mantiene sin bajar, cortar el entrenamiento. Aprovechando esto, vamos a entrenar una mayor cantidad de epochs, y aumentar el batch_size respecto al modelo anterior.

In [ ]:
%%time
# Entrenamiento del modelo
es = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
history = model_nr.fit(x_train,y_train,epochs=100, batch_size=128, callbacks=[es], verbose=True)

In [ ]:
pred_red_r = model_nr.predict(x_test)

print_score(y_test, pred_red_r)

In [ ]:
rn5_cm = confusion_matrix_by_client(y_test,pred_red_r)
sns.heatmap(rn5_cm, cmap='Blues', annot=True, fmt='g')
plt.xlabel('Predicted')
plt.ylabel('True')

In [ ]:
plot_hist(history)

Se ve como a partir de las 20 epochs no hay mucha ganancia en accuracy y el modelo no aprende nada nuevo, por lo que podemos reducir la cantidad de epochs

### Ensamble en cascada

Creamos un nuevo clasificador en base a los modelos ya generados y a la red neuronal. Funciona de la siguiente manera:

Sea X un cliente

X -> RED(X) = P("X NO ES DEUDOR") < ESPECTRO -> RF(X) = P("X NO ES DEUDOR") < ESPECTRO -> XGB(X) = P("X NO ES DEUDOR") < ESPECTRO -> SVM(X) = P("X NO ES DEUDOR") < ESPECTRO -> X ES DEUDOR (PREDICT TARGET = 1)

Si algun modelo predice una probabilidad mayor que el espectro de que ese cliente no sea deudor, se lo considera no deudor (target = 0)

In [ ]:
from sklearn.calibration import CalibratedClassifierCV

def predict_clients(x_test)
    arbol_probas = arbol.predict_proba(x_test)
    xgb_probas = xgb_model.predict_proba(x_test)
    calibrated = CalibratedClassifierCV(svm_model)
    calibrated.fit(x_train, y_train)
    svm_probas = calibrated.predict_proba(x_test)
    red_probas = red_2h1d_v2.predict(x_test)
    
    for i in range(len(red_probas)):
        red_probas[i] = 1- red_probas[i] # para quedarnos con la proba de que target == 0 y no == 1
    #me quedo solo con las probabilidades de target == 0
    return [red_probas, arbol_probas[0], xgb_probas[0], svm_probas[0]]

#recibe x_test e y_test de todos los clientes y un espectro de no deudor
def cascade_ensemble(x_test, spectre):
    customers = x_test['customer_ID'].values
    predicts = predict_clients(x_test)
    df_predicts = []
    
    #df_predicts es una lista con dataframes en el siguiente orden:
    #DF[0]: |customer_ID|proba_red| ; #DF[1]: |customer_ID|proba_arbol| ; #DF[2]: |customer_ID|proba_xgb| ...
    #cada probabilidad es por registro, por lo que repite clientes
    for i in range(len(predicts)):
        df_predicts[i] = pd.DataFrame(columns = ['customer_ID', 'target_proba'])
        for j in range(len(customers)):
            df_predict[i].append({'customer_ID': customers[j], 'target_proba': predicts[i][j]})
            
    #para cada dataframe hago un aggregate de las probabilidades agrupado por customer_ID, 
    #me queda un unico cliente por dataframe
    for i in range(len(df_predicts)):
        df_predicts[i] = df_predicts[i][['customer_ID', 'target_proba']].groupby('customer_ID').agg({'target_proba':custom_predict})
    
    customers = x_test['customer_ID'].unique().values
    cascading_result = pd.DataFrame(columns = ['customer_ID', 'target'])
    
    #para cada dataframe, aplico cascada por cliente
    for i in range(len(customers)):
        target0=False
        for j in df_predicts:
            current_df = df_predicts[j]
            #busco el customer id, busco la unica proba que deberia aparecer
            result = current_df[current_df['customer_ID']==customers[i]]['target_proba'].values[0]
            # si la proba de que sea 0 es mayor al spectre, entonces confirmamos que es 0,
            # sino se lo pasamos al siguiente modelo...
            # si no se cumple para ningun modelo, asumimos que es deudor (1)
            if (result > spectre):
                target0 = True
                cascading_result.append(cascading_result.append({'customer_ID': customers[i], 'target' : 0}, 
                ignore_index = True));
                break
        if not target0:
            results.append(results.append({'customer_ID': customers[i], 'target' : 1}, 
                ignore_index = True));
    return cascading_result

def cascade_metrics(x_test, y_test, cascade_results):
    test_real = x_test
    test_real['target'] = y_test
    test_real = test_real[['customer_ID', 'target']].groupby('customer_ID').agg({'target':custom_predict})
    
    acc, recall, f1 = metrics_by_client(test_real['target'].values, cascade_results['target'].values)
    
    return acc, recall, f1

def cascade_cm(x_test, y_test, cascade_results):
    test_real = x_test
    test_real['target'] = y_test
    test_real = test_real[['customer_ID', 'target']].groupby('customer_ID').agg({'target':custom_predict})
    
    return confusion_matrix(test_real['target'].values, cascade_results['target'].values)
    
def predict_cascade(x_test, spectre):
    #dataframe con el customer id y la prediccion de target
    cascade_results = cascade_ensemble(x_test, spectre)
    
    cascade_acc, cascade_recall, cascade_f1 = cascade_metrics(x_test, y_test, cascade_results)

    print(f"Metricas con spectre={spectre}:")
    print("La precision es {}".format(cascade_acc))

    print("El recall es {}".format(cascade_recall))

    print("F1 es {}".format(cascade_f1))
    
    cascada_cm = cascade_cm(x_test_copy, y_test, cascade_results)
    sns.heatmap(cascada_cm, cmap='Blues', annot=True, fmt='g')
    plt.title("Confusion Matrix spectre={}".format(spectre))
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

Vamos a probar con varios valores de spectre y analizar los resultados:

Probamos desde valores altos hasta valores más bajos y viendo cómo se comporta el ensamble en cascada. Vamos desde 0.99 hasta 0.51

In [ ]:
spectre_values = [0.99, 0.9, 0.85, 0.8, 0.7, 0.51]
for s in spectre_values:
    predict_cascade(x_test, s)

A medida que vamos disminuyendo el valor del espectro, vemos como nuestro modelo predice mejor, disminuyendo la cantidad de Falsos Positivos. Esto tiene sentido, porque quiere cuanto más seguros tenemos que estar para dar una predicción de 0, menor cantidad de 0s voy a predecir, pero los pocos que prediga, la certeza será mayor. Como se aprecia, nuestros modelos no nos garantizan probabilidades muy altas para valores de espectro altas, y a medida que vamos cediendo confianza para la detección de no deudores, mayor es la cantidad de Falsos Negativos que van apareciendo.

Para un valor de spectre=0.51, esto significa que vamos a fijarnos en el modelo 1 si la predicción está más cerca del 0 que del 1. Si está cerca del 0 entonces confirmamos que es 0, caso opuesto se lo pasamos al siguiente modelo y así sucesivamente. Si al menos uno de esos modelos predice que está cerca del 0, entonces confirmaríamos que no es deudor. Al ser 51% una probabilidad muy cercana a la otra clase, perdemos la confianza en nuestra predicción y por eso es que se aprecia mayor cantidad de Falsos Negativos.

Un valor razonable podría ser spectre=0.8, es decir que tenemos que estar 80% seguros de que nuestra predicción sea 0 (no deudor) antes de confirmarla como tal. Si no estamos seguros, se lo pasamos al siguiente modelo y así hasta tener la certeza que sea no deudor. Caso opuesto, asumimos que es deudor no pago.

In [ ]:
#Predict por cascada, agregar predecir por cliente
cascade_results = cascade_ensemble(x_test, 0.8)

#cascade results deberia dar un resultado por cliente o por resumen?
cascade_acc, cascade_recall, cascade_f1 = metrics_by_client(y_test, cascade_results)

print(f"Metricas con spectre={0.8}:")
print("La precision es {}".format(cascade_acc))

print("El recall es {}".format(cascade_recall))

print("F1 es {}".format(cascade_f1))

cascada_cm = confusion_matrix_by_client(y_test,cascade_results)
sns.heatmap(cascada_cm, cmap='Blues', annot=True, fmt='g')
plt.title("Confusion Matrix spectre={}".format(0.8))
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

## Conclusión

Resumimos a continuación las métricas de cada modelo:

**Random Forest**  
La precision es 0.8507193794570249  
El recall es 0.8507193794570249  
F1 es 0.8504343815456924  

**XGBoost**  
La precision es 0.8569998748905292  
El recall es 0.8569998748905292  
F1 es 0.856881445682361  

**SVM**  
La precision es 0.8608032028024521  
El recall es 0.8608032028024521  
F1 es 0.8606546296803764  

**Ensamble (Voting classifier con RandomForest, XGBoost y SVM)**  
La precision es 0.8596021518828976  
El recall es 0.8596021518828976  
F1 es 0.8594262414104039  


***Redes neuronales***  

**Modelo 1 (Red base)**  
La precision es 0.8625297134993118  
El recall es 0.8625297134993118  
F1 es 0.8623243443521068  

**Modelo 2 (Red base v2)**  
La precision es 0.8626298010759414  
El recall es 0.8626298010759414  
F1 es 0.8624413655524797  

**Modelo 3 (Hidden layer)**  
La precision es 0.8619041661453772  
El recall es 0.8619041661453772  
F1 es 0.8617301517248295  

**Modelo 4 (Hidden layer v2)**  
La precision es 0.863830851995496  
El recall es 0.863830851995496  
F1 es 0.8636409981984825  

**Modelo 5**  
La precision es 0.8635556111597648  
El recall es 0.8635556111597648  
F1 es 0.8632714630832788  


**Ensamble (cascading con una red neuronal, Random Forest, XGBoost y SVM)**  
La precision es 0.826848492430877  
El recall es 0.826848492430877  
F1 es 0.8228160141119752  

Graficamos a continuación una comparativa entre las distintas redes neuronales

In [ ]:
F1_redes = {"Modelo 1 (Red base)": 0.8623243443521068, "Modelo 2 (Red base v2)": 0.8624413655524797, \
            "Modelo 3 (Hidden layer)": 0.8617301517248295, "Modelo 4 (Hidden layer v2)": 0.8636409981984825, \
            "Modelo 5": 0.8632714630832788}

F1_set = pd.DataFrame(list(F1_redes.items()))
F1_set.columns = ["Red", "F1"]
F1_set

In [ ]:
plt.figure(figsize=(7,12))
ax = sns.barplot(x="Red", y="F1", data=F1_set ,alpha=0.5, orient='v')
plt.xticks(rotation=90)
plt.show()

Cortamos el gráfico para que se vea mejor la diferencia

In [ ]:
plt.figure(figsize=(7,6))
ax = sns.barplot(x="Red", y=F1_set.F1 - 0.86, data=F1_set ,alpha=0.5, orient='v')
plt.xticks(rotation=90)
ax.set_ylabel("F1 (0.86 + y)")
plt.show()

Si bien las 5 redes dieron resultados similares y buenos (> 0.86), Los modelos 4 y 5 fueron los que mejor resultado tuvieron.

Graficamos a continuación una comparativa entre los distintos modelos, para la red neuronal usamos al modelo 4 (que además es la red utilizada en el ensamble de tipo cascada debido a su buen resultado)

In [ ]:
F1_modelos = {"Random Forest": 0.8504343815456924, "XGBoost": 0.856881445682361, \
              "SVM": 0.8606546296803764, "Voting": 0.8594262414104039, \
              "Red neuronal": 0.8636409981984825, "Cascading": 0.8228160141119752}

F1_set = pd.DataFrame(list(F1_modelos.items()))
F1_set.columns = ["Modelo", "F1"]
F1_set

In [ ]:
plt.figure(figsize=(7,12))
ax = sns.barplot(x="Modelo", y="F1", data=F1_set ,alpha=0.5, orient='v')
plt.xticks(rotation=90)
plt.show()

Vamos a cortar el gráfico para que se puede ver mejor la diferencia

In [ ]:
plt.figure(figsize=(7,6))
ax = sns.barplot(x="Modelo", y=F1_set.F1 - 0.8, data=F1_set ,alpha=0.5, orient='v')
plt.xticks(rotation=90)
ax.set_ylabel("F1 (0.8 + y)")
plt.show()

Aquí se ve cómo los mejores modelos fueron la red neuronal y la Support Vector Machine. También se puede ver que Voting fue mejor que Random Forest y XGBoost, y que Cascading fue el peor estimador a pesar de estar hecho con los anteriores.